## Este Notebook tem como objetivo:
* ### extrair o texto das páginas de cada semente, de 10 sites. São eles:
  1. Syngenta
  2. NK sementes (também são da Syngenta)
  3. TMG
  4. Don Mario cerrado
  5. Don Mario Sul
  6. Brasmax Cerrado
  7. Brasmax Sul
  8. Nidera
  9. Brevant
  10. KWS

In [1]:
url_1 = 'https://www.portalsyngenta.com.br/sementes/syngenta-soja'
url_2 = 'https://www.portalsyngenta.com.br/sementes/nk-soja'
url_3 = 'https://www.tmg.agr.br/ptbr/cultivares/soja'
url_4 = 'https://www.donmario.com/pt-br/catalogo-cerrado/'
url_5 = 'https://www.donmario.com/pt-br/catalogo-sul/'
url_6 = 'https://www.brasmaxgenetica.com.br/cultivar-regiao-cerrado/'
url_7 = 'https://www.brasmaxgenetica.com.br/cultivar-regiao-sul/'
url_8 = 'https://www.niderasementes.com.br/soja/'
url_9 = 'https://www.brevant.com.br/produtos/soja.html'
url_10 = 'https://www.kws.com/br/pt/produtos/soja/portfolio-completo-soja/'